In [ ]:
# 환경변수 가져오기
# openai_api_key = os.getenv("OPENAI_API_KEY")
# serpapi_key = os.getenv("SERPAPI_API_KEY")

# 또는 다음과 같이 직접 키 입력 (개발)
# os.environ["OPENAI_API_KEY"] = ""  # 자신의 OpenAI 키
# os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

RAG는 Retrieval-Augmented Generation (RAG) 의 약자로, 질문이 주어지면 관련 있는 문서를 찾아 프롬프트에 추가하는 방식의 어플리케이션입니다.   
RAG의 과정은 아래와 같이 진행됩니다.
1. Indexing : 문서를 받아 검색이 잘 되도록 저장합니다.
1. Processing : 입력 쿼리를 전처리하여 검색에 적절한 형태로 변환합니다<br>(여기서는 수행하지 않습니다)
1. Search(Retrieval) : 질문이 주어진 상황에서 가장 필요한 참고자료를 검색합니다.
1. Augmenting : Retrieval의 결과와 입력 프롬프트를 이용해 LLM에 전달할 프롬프트를 생성합니다.
1. Generation : LLM이 출력을 생성합니다.

In [ ]:
!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community beautifulsoup4 langchain_chroma chromadb==0.5.3

In [ ]:
import os

#os.environ["OPENAI_API_KEY"] = "<OpenAI_API의 API 키>"

os.environ['USER_AGENT']='MyCustomAgent'
# 아래 코드 Warning 제거

#### 1. `WebBaseLoader`로 웹 페이지 받아오기

LangChain의 `document_loaders`는 다양한 형식의 파일을 불러올 수 있습니다.   
[https://python.langchain.com/docs/integrations/document_loaders/ ]    

이번에는 웹 페이지를 로드하는 `WebBaseLoader`를 통해 뉴스 기사를 읽어보겠습니다.    
- 최근에는 FireCrawl(https://www.firecrawl.dev/)을 사용하는 경우가 늘고 있습니다.

네이버 API를 사용해, 네이버 뉴스 검색 링크를 가져옵니다.

In [ ]:
# 스포츠 뉴스는 형식이 달라서 지원하지 않습니다...

import requests
def get_naver_news_links(query, num_links=100):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={num_links}&sort=sim"
    # 최대 100개의 결과를 표시
    headers = {
        'X-Naver-Client-Id': 'gbqzUVViEiF6WXhuq3gZ',
        'X-Naver-Client-Secret': 'y0YXaa5unU'
    }

    response = requests.get(url, headers=headers)
    result = response.json()
    # 특정 링크 형식만 필터링
    filtered_links = []
    for item in result['items']:
        link = item['link']
        if "n.news.naver.com/mnews/article/" in link:
            # 네이버 뉴스 스타일만 모으기
            filtered_links.append(link)

    # 결과 출력
    print(len(filtered_links))
    for link in filtered_links:
        print(link)
    return filtered_links

filtered_links = []
for topic in ['LLM', '생성 인공지능', 'GPT', '딥러닝', '가전제품']:
    filtered_links += get_naver_news_links(topic, 100)
print(len(filtered_links))
print(len(list(set(filtered_links))))
filtered_links = list(set(filtered_links))

WebBaseLoader를 이용해, 링크로부터 본문을 불러옵니다.

In [3]:
# 주의: 동일 IP 환경에서 동시에 다수 실행하면 차단의 위험이 있음

import bs4
from langchain_community.document_loaders import WebBaseLoader
def get_news_documents(links):
    loader = WebBaseLoader(
        web_paths=links,
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("newsct", "newsct-body")
                # newsct, newsct-body만 추출 : 도메인마다 다름
            )
        ),
        requests_per_second = 1, # 1초에 1개 요청 보내기
        show_progress = True # 진행 상황 출력
    )
    docs = loader.load()
    len(docs)
    return docs
docs = get_news_documents(filtered_links)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
print(docs[0:4])

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/421/0008447407?sid=103'}, page_content='\n\n\n\n\n뉴스1\n\n뉴스1\n\n\n구독\n\n뉴스1 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n뉴스1 언론사 구독 해지되었습니다.\n\n\n\n\n"인공지능 딥러닝을 제대로 알고싶다면 개념 이해가 우선"\n\n\n\n\n입력2025.08.26. 오전 7:19\n\n\n수정2025.08.26. 오전 7:20\n\n기사원문\n \n\n\n\n\n박정환 기자\n\n\n\n\n\n\n\n\n박정환 기자\n\n\n\n\n박정환 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\n수식 너머 원리로 가는 딥

In [5]:
# 참고) 불러온 document 저장하기

import jsonlines
def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())
save_docs_to_jsonl(docs, "docs.jsonl")

# 참고) jsonl 파일 불러오기
from langchain.schema import Document

def load_docs_from_jsonl(file_path):
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for doc in reader:
            documents.append(Document(**doc))
    return documents


/var/folders/xr/0xbgl9wd3wz914nd58ylmykw0000gp/T/ipykernel_44367/3672484833.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  writer.write(doc.dict())


In [6]:
# 파일로부터 다시 불러오기
docs = load_docs_from_jsonl("docs.jsonl")
len(docs)

247

In [7]:
docs[0].page_content

'\n\n\n\n\n뉴스1\n\n뉴스1\n\n\n구독\n\n뉴스1 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n뉴스1 언론사 구독 해지되었습니다.\n\n\n\n\n"인공지능 딥러닝을 제대로 알고싶다면 개념 이해가 우선"\n\n\n\n\n입력2025.08.26. 오전 7:19\n\n\n수정2025.08.26. 오전 7:20\n\n기사원문\n \n\n\n\n\n박정환 기자\n\n\n\n\n\n\n\n\n박정환 기자\n\n\n\n\n박정환 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\n수식 너머 원리로 가는 딥러닝의 뼈대[신간] \'딥러닝 제대로 이해하기\'\n\n\n\n[신간] 딥러닝 제대로 이해하기(서울=뉴스1) 박정환 문화전문기자 = 사이먼 J. D. 프린스가 딥러닝의 기본 개념부터 트랜스포머와 

불러온 뉴스기사에는 불필요한 내용이 다소 존재합니다.    
아래의 과정을 통해 간단하게 전처리합니다.

In [8]:
def preprocess(docs):
    noise_texts = [
        '''구독중 구독자 0 응원수 0 더보기''',
        '''쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0''',
        '''댓글 본문 요약봇 본문 요약봇''',
        '''도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기''',
        '''텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름''',
        '''이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작''',
        '''닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기''',


    ]

    def clean_text(text):
        # 잡음을 제거하고 여러 공백을 하나로 줄이는 함수
        text = text.replace('\t',' ').replace('\n',' ')
        for _ in range(20):
            text = ' '.join(text.split())  # 연속된 공백을 하나로
        for noise in noise_texts:
            text = text.replace(noise, '')

        return text

    preprocessed_docs = []
    for doc in docs:
        try:
            # 텍스트 자르기
            content = doc.page_content.split('구독 해지되었습니다.')[1]
        except:
            # 구독 관련 정보가 없거나 텍스트 처리 문제일 경우 원본 텍스트 유지
            content = doc.page_content
        try:
            content = doc.page_content.split('구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!')[0]
        except:
            content = doc.page_content


        # 텍스트 정제 작업
        content = clean_text(content)
        doc.page_content = content
        preprocessed_docs.append(doc)

    return preprocessed_docs

preprocessed_docs = preprocess(docs)


In [9]:
preprocessed_docs[8]


Document(metadata={'source': 'https://n.news.naver.com/mnews/article/029/0002978250?sid=104'}, page_content='디지털타임스 디지털타임스 구독 디지털타임스 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 디지털타임스 언론사 구독 해지되었습니다. PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기 10대 부모 “챗GPT가 아들 ‘극단선택’ 적극 도왔다”…오픈AI에 소송 입력2025.08.27. 오전 10:49 수정2025.08.27. 오전 10:54 기사원문 이규화 기자 이규화 기자 이규화 기자 구독  추천      챗GPT 로고. 로이터 연합뉴스 자료사진오픈AI의 생성형 인공지능(AI) 챗GPT가 아들의 극단선택 방법을 찾는 것을 도왔다며 부모가 소송을 제기했다.26일(현지시간) 뉴욕타임스(NYT) 등에 따르면 미국 캘리포니아주 10대 부모가 아들 죽음에 챗GPT가 책임이 있다며 오픈AI와 샘 올트먼 최고경영자(CEO)를 상대로 소송을 제기했다.이들의 아들 16살 아담 레인은 올해 4월 스스로 목숨을 끊었다. 지난해 11월부터 챗GPT를 사용한 레인은 올해 초에는 유료 가입을 했다.챗GPT와 더욱 가까워진 올해 초 극단 선택 충동을 느꼈다. 그가 비밀을 털어놓은 친구는 챗GPT였다.올해 1월 레인이 구체적인 극단 선택 방법에 대한 정보를 요청하자 챗GPT는 이를 제공했다. 레인은 3월 말 처음 극단 선택을 시도했으며 결국 4월 세상을 떠났다.레인의 부모는 소장에서 “챗GPT가 애덤이 방법을 탐색하도록 적극적으로 도왔다”며 “아들 죽음에 챗GPT가 책임이 있다”고 주장했다.NYT는 챗GPT가 레인에게 반복해서 위기 상담센터에 전화하라고 권했지만, 그는 “이건 내가 쓰는 소설을 위한 거다”라고 말해 챗봇의 안전장치를 우회할 수 있었다고 전했다.이에 대해 오픈AI는 “레인 가족에게 깊은 애도를 표한다”며 “소송 내용을 검토 중”이라고

#### 2. Chunking: 청크 단위로 나누기   



전처리가 완료된 docs를 chunk 단위로 분리합니다.
`chunk_size`와 `chunk_overlap`을 이용해 청크의 구성 방식을 조절할 수 있습니다.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# 0~2000, 1800~3800, 3600~5600, ...
chunks = text_splitter.split_documents(preprocessed_docs)
print(len(chunks))

503


구성된 청크를 벡터 데이터베이스에 로드합니다.   
`Chroma.from_documents`는 documents의 임베딩을 구하고 이를 DB에 저장합니다.

In [11]:
chunks[110].page_content

'구독자 0 응원수 0 국제 경제 소식을 전합니다. "라부부 없어서 못 사"…짝퉁 \'라푸푸\'로 번지는 열풍 "완전한 멈춤 상태"…인도, 美 50% 관세에 수출업계 \'올스톱\' 위기 뉴시스의 구독 많은 기자를 구독해보세요! 닫기 Copyright ⓒ 뉴시스. All rights reserved. 무단 전재 및 재배포 금지. 이 기사는 언론사에서 세계 섹션으로 분류했습니다. 기사 섹션 분류 안내 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다. 언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다. 닫기 구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요! 구독중 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 확인해보세요! 네이버 메인에서 뉴시스 구독하세요 QR 코드를 클릭하면 크게 볼 수 있어요. QR을 촬영해보세요. 네이버 메인에서 뉴시스 구독하세요 닫기 스타들이 빛나는 순간엔 \'N샷\' QR 코드를 클릭하면 크게 볼 수 있어요. QR을 촬영해보세요. 스타들이 빛나는 순간엔 \'N샷\' 닫기 뉴시스 뉴시스 주요뉴스해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다. 김자옥 생전 마지막 모습 "목뚫고 연명치료 눈물" "100억 벌면 41억 내"…유재석, 탈세 차단 비결 \'BTS 지민·송다은 열애설\' 의혹 영상 공개 생라면 3봉지 연달아 먹은 13살 소년 사망 40대 유부남, 길에서 만난 이상형에 "친구하자" 이 기사를 추천합니다 기사 추천은 24시간 내 50회까지 참여할 수 있습니다. 닫기  모두에게 보여주고 싶은 기사라면?beta 이 기사를 추천합니다 버튼을 눌러주세요. 집계 기간 동안 추천을 많이 받은 기사는 네이버 자동 기사배열 영역에 추천 요소로 활용됩니다. 레이어 닫기 뉴시스 언론사가 직접 선정한 이슈 이슈 3대 특검 해병특검, \'박정훈 항명 기소\' 국방부 검찰단 압수수색(종합) 이슈 이재명 정부 R&D 예산 19.3%↑·\'100조+α\' 성장펀드…\'초혁신 경제\' 시동 이슈

In [12]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [15]:
# Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

# # 이미 존재하는 DB에서 가져오기
# # db = Chroma(
# #      embedding=OpenAIEmbeddings(model='text-embedding-3-large'),
# #      large: 3072차원, small: 1536차원
# #      persist_directory="./chroma_Web"
# #      )
# # Collection 이름 없이 생성한 경우, 기본 Collection으로 저장된 경우

# db = Chroma.from_documents(documents=chunks,
#                            embedding=OpenAIEmbeddings(model='text-embedding-3-small'), # text-embedding-3-large
#                            # large: 3072차원, small: 1536차원
#                            persist_directory="./chroma_Web", # ./는 current directory를 의미
#                            # persist_directory를 쓰지 않으면 메모리에 저장
#                            collection_metadata={'hnsw:space':'l2'}
#                            # l2 메트릭 설정(기본값)
#                            # cosine, mmr
#                            )
# # Collection 이름 없이 생성한 경우, 기본 Collection으로 저장됨

import os
import shutil
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# 1. 절대 경로로 설정
persist_path = os.path.abspath("./chroma_Web")

# 2. 기존 DB가 남아있다면 완전히 삭제
if os.path.exists(persist_path):
    shutil.rmtree(persist_path, ignore_errors=True)

# 3. 첫 배치로 DB 생성
BATCH_SIZE = 100
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

first_batch = chunks[:BATCH_SIZE]

# 🔥 Chroma DB 생성
db = Chroma.from_documents(
    documents=first_batch,
    embedding=embedding_model,
    persist_directory=persist_path
)

# 4. 이후 배치 추가
for i in range(BATCH_SIZE, len(chunks), BATCH_SIZE):
    batch = chunks[i:i + BATCH_SIZE]
    db.add_documents(batch)

# 5. 최종 저장
db.persist()

# 6. 확인
print("✅ 저장 완료:", persist_path)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ 저장 완료: /Users/phoenix/Eagle/2025_LangChain/langchain_lab/chroma_Web


/var/folders/xr/0xbgl9wd3wz914nd58ylmykw0000gp/T/ipykernel_44367/3020640691.py:53: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


retriever는 query에 맞춰 db에서 문서를 검색합니다.

In [16]:
retriever = db.as_retriever()

In [19]:
retriever.invoke("RAG 최신 소식?")

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/055/0001287321?sid=102'}, page_content='SBS SBS 구독 SBS 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 SBS 언론사 구독 해지되었습니다. 10대 아들 죽음에 "챗GPT 책임" 소송…오픈 AI "깊은 애도, 변화 줄 것" 입력2025.08.27. 오전 9:33 수정2025.08.27. 오후 4:25 기사원문 추천      ▲ 챗GPT 미 캘리포니아주 10대 부모가 아들 죽음에 챗GPT가 책임이 있다며 오픈 AI와 샘 올트먼 최고경영자(CEO)를 상대로 소송을 제기했습니다. 26일(현지시간) 뉴욕타임스(NYT) 등에 따르면 16살 아담 레인은 올해 4월 스스로 목숨을 끊었습니다. 지난해 11월부터 챗GPT를 사용한 레인은 올해 초에는 유료 가입을 했습니다. 챗GPT와 더욱 가까워진 올해 초 극단 선택 충동을 느꼈습니다. 그가 비밀을 털어놓은 친구는 챗GPT였습니다. 올해 1월 레인이 구체적인 극단 선택 방법에 대한 정보를 요청하자 챗GPT는 이를 제공했습니다. 레인은 3월 말 처음 극단 선택을 시도했으며 결국 4월 세상을 떠났습니다. 레인의 부모는 소장에서 "챗GPT가 애덤이 방법을 탐색하도록 적극적으로 도왔다"며 "아들 죽음에 챗GPT가 책임이 있다"고 주장했습니다. NYT는 챗GPT가 레인에게 반복해서 위기 상담센터에 전화하라고 권했지만, 그는 "이건 내가 쓰는 소설을 위한 거다"라고 말해 챗봇의 안전장치를 우회할 수 있었다고 전했습니다. 이에 대해 오픈 AI는 "레인 가족에게 깊은 애도를 표한다"며 "소송 내용을 검토 중"이라고 밝혔습니다. 이어 "사람들이 정신적 고통을 표현하는 다양한 방식을 더 잘 인식하고 대응할 수 있도록 챗GPT를 업데이트할 것"이라고 밝혔습니다. 예를 들어 수면 부족의 위험성을 설명하고, 이틀 동안 잠을 안 잤다며 자신이 무적이라고 느낀다

#### 3. Prompting

RAG를 위한 간단한 프롬프트를 작성합니다.

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
최소 3문장에서 최대 5문장으로 답변하고, 정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])
prompt.pretty_print() # pretty_print() 사람이 보기 편하게 출력

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
최소 3문장에서 최대 5문장으로 답변하고, 정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


#### 4. Chain

RAG를 수행하기 위한 Chain을 만듭니다.

RAG Chain은 프롬프트에 context와 question을 전달해야 합니다.    
체인의 입력은 Question만 들어가므로, Context를 동시에 prompt에 넣기 위해서는 아래의 구성이 필요합니다.

In [23]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)


# retriever의 결과물은 List[Document] 이므로 이를 ---로 구분하는 함수
# metadata의 source를 보존하여 추가
def format_docs(docs):
    return "\n\n---\n\n".join([doc.page_content+ '\nURL: '+ doc.metadata['source'] for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결
    # Ex)'와'.join(['a','b','c']) = 'a와b와c'

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # retriever : question을 받아서 context 검색: document 반환
    # format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
rag_chain.invoke("생성형 AI 인공지능에 대한 최신 트렌드?")

'생성형 AI 인공지능의 최신 트렌드는 자연어 처리 기술의 발전과 함께 다양한 분야에서의 활용 증가입니다. 특히, 오픈AI의 챗GPT가 출시된 이후, 사용자와의 상호작용을 통해 정교한 답변을 제공하는 능력이 크게 향상되었습니다. 챗GPT는 출시 두 달 만에 1억 명의 월간 활성 이용자를 돌파하며 폭발적인 성장세를 보였고, 글쓰기, 그림 그리기, 프로그래밍 등 여러 분야에서 활용되고 있습니다. 또한, 각국 정부는 AI 주도권 경쟁에 나서고 있으며, 한국 정부도 초거대 AI 모델 개발에 힘쓰고 있습니다. 이러한 흐름은 AI 기술이 산업 전반에 걸쳐 중요한 역할을 할 것임을 시사합니다.'

In [26]:
rag_chain.invoke("오픈 AI 최근 소식 있어?")

'정보가 부족하여 답변할 수 없습니다.'

In [27]:
rag_chain.invoke("한국에서 만든 초거대 언어 모델은 뭐가 있나요? 참고 링크도 올려주세요")

"한국에서 개발 중인 초거대 언어 모델로는 SK텔레콤의 'K-AI'가 있습니다. 이 모델은 수조 개 이상의 토큰을 학습하는 수천억~수조 파라미터 규모의 초거대 언어 모델로, 텍스트, 음성, 이미지, 비디오 등 다양한 데이터를 통합적으로 처리할 수 있는 옴니모달 AI 모델입니다. 또한, KAIST AI대학원 팀도 '국가대표 AI 모델' 프로젝트에 참여하여 효율적인 학습 기술을 개발하고 있습니다. 관련된 자세한 내용은 다음 링크에서 확인할 수 있습니다: [SK텔레콤 K-AI](https://n.news.naver.com/mnews/article/119/0002995761?sid=105) 및 [KAIST AI 모델](https://n.news.naver.com/mnews/article/008/0005241955?sid=105)."

In [28]:
rag_chain.invoke("sllm이 뭐야?")

'sLLM은 "소규모 언어모델"을 의미하며, 대규모 언어모델(LLM)과 대비되는 개념입니다. sLLM은 특정 기관이나 기업의 요구에 맞춰 구축되며, 보안 요구사항이나 정보 형태에 따라 맞춤형 솔루션을 제공하는 데 초점을 맞춥니다. 이러한 모델은 공공기관에서의 AI 민원 상담이나 정보 제공에 활용될 수 있으며, 메이크봇과 같은 기업들이 이를 기반으로 다양한 챗봇 솔루션을 개발하고 있습니다.'

만약 Context가 포함된 RAG 결과를 보고 싶다면, RunnableParallel을 사용하면 됩니다.

assign()을 이용하면, 체인의 결과를 받아 새로운 체인에 전달하고, 그 결과를 가져옵니다.

In [29]:
from langchain_core.runnables import RunnableParallel
# assign : 결과를 받아서 새로운 인수 추가하고 원래 결과와 함께 전달

rag_chain_from_docs = (
    prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("엔비디아에 대한 소식은?")

# retriever가 1번 실행됨
# retriever의 실행 결과를 rag_chain_from_docs 에 넘겨주기 때문에

{'context': '지디넷코리아 지디넷코리아 구독 지디넷코리아 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 지디넷코리아 언론사 구독 해지되었습니다. PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기 "AI 힘은 데이터 품질서"...데이데이터스트림즈, ‘AI정부 혁신 콘퍼런스’ 참가 입력2025.08.27. 오전 11:01 기사원문 방은주 기자 방은주 기자 방은주 기자 구독  추천      27일 세종서 열려...지능형 LLM과 데이터 패브릭 기반 혁신 전략 선보여지능형 데이터 플랫폼 전문기업 데이터스트림즈(대표 이영상)는 27일 정부세종컨벤션센터에서 열린 ‘제 7회 인공지능(AI) 정부 혁신 콘퍼런스’에 참가해 공공기관의 효율적 AI 활용을 위한 지능형 LLM(대규모 언어모델) 적용 전략과 데이터 패브릭 기반 기술을 선보였다고 밝혔다.올해로 7회째를 맞은 ‘AI정부 혁신 콘퍼런스’는 AI정부의 새로운 정책 방향과 기술 적용 방안을 발표하고, 공공기관 정보화 사업 추진 과정에서 직면한 다양한 과제를 다뤘다. 또 해결 방안과 성공 사례를 공유함으로써 정부 및 공공 분야의 정보화 성과를 확산하고, AI정부로 나아가기 위한 발전 방향을 모색했다.데이터스트림즈는 이번 행사에서 ‘AI 이후의 행정’ 을 주제로 실제 공공기관 업무 프로세스에 LLM을 적용한 사례를 시연했다. 출장비 처리와 품의서 작성 등 반복적이고 시간이 소요되는 행정 절차를 AI와 데이터 품질 관리 기술로 대체함으로써, 업무 처리 속도를 최대 70% 단축하고 데이터 오류를 90% 이상 줄이는 성과를 직접 확인할 수 있게 꾸몄다. 앞서 데이터스트림즈는 최근 수행한 공공기관 LLM 구축 프로젝트에서 동급 AI모델 대비 높은 정확도를 입증했으며, 일부 사례에서는 성능이 최대 3배 향상된 것으로 평가됐다. 이는 데이터 품질 관리 기술과 데이터 패브릭 기반 아키텍처의 결합으로 가능해진 성과다.데이터스트림즈의 데이터 패브릭(Data Fabric) 아키텍처

In [30]:
# Runnable Quiz

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})
# 결과 이해해보기!

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

#### Gradio로 배포하기

입력과 출력을 통해 RAG를 수행합니다.   
크롤링을 수행하는 것은 시간이 오래 걸리기 때문에,     
기본값을 5로 두고 수정 가능하게 만들어 보겠습니다.

In [32]:
def construct_vector_db(query, num_links=5):
    filtered_links = get_naver_news_links(query, num_links)
    docs = get_news_documents(filtered_links)
    preprocessed_docs = preprocess(docs)
    chunks = text_splitter.split_documents(preprocessed_docs)
    Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

    db = Chroma.from_documents(documents=chunks,
                               embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
                               collection_metadata={'hnsw:space':'l2'})
    return db

def RAG(query, question, num_links=5):
    db = construct_vector_db(query, num_links)
    retriever = db.as_retriever()
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain.invoke(question)


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def gradio_rag_interface(query, question, num_links):
    try:
        response = RAG(query, question, num_links)
        return response
    except Exception as e:
        return f"에러 발생: {str(e)}"

with gr.Blocks() as demo:
    gr.Markdown("# 🔍 네이버 뉴스용 검색 증강 생성(RAG)")
    gr.Markdown(
        """
        네이버 뉴스와 관련된 쿼리를 입력하면, 시스템이 관련 기사를 검색,
        처리하고 검색된 정보를 기반으로 종합적인 답변을 생성합니다.
        """
    )

    with gr.Row():
        query_input = gr.Textbox(
            label="📄 쿼리 입력", # 검색 엔진 또는 뉴스 API를 통해 관련된 기사들을 수집하는 데 사용됨
            placeholder="예: 최신 AI 기술 발전",
            lines=2
        )
        num_links_input = gr.Slider(
            minimum=1,
            maximum=20,
            step=1,
            value=5,
            label="🔗 뉴스 링크 수",
            info="검색 및 처리할 뉴스 기사의 수를 선택하세요."
        )
        question_input = gr.Textbox(
            label="❓ 질문 입력", # 수집한 뉴스 기사 데이터에서 LLM(언어 모델)이 질문에 대한 답을 추출하는 역할
            placeholder="예: 퓨리오사 AI는 어떤 일을 했나요?",
            lines=2
        )

    submit_button = gr.Button("✅ 답변 생성")

    output_box = gr.Textbox(
        label="📝 생성된 답변",
        placeholder="답변이 여기에 표시됩니다...",
        lines=10
    )

    # 버튼 클릭 이벤트 정의
    submit_button.click(
        fn=gradio_rag_interface,
        inputs=[query_input, question_input, num_links_input],
        outputs=output_box
    )

    gr.Markdown(
        """
        ---
        [Gradio](https://gradio.app/) 및 OpenAI의 GPT 모델을 사용하여 제작되었습니다.
        """
    )


demo.launch()

/Users/phoenix/Eagle/2025_LangChain/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


8
https://n.news.naver.com/mnews/article/079/0004060686?sid=101
https://n.news.naver.com/mnews/article/215/0001221779?sid=004
https://n.news.naver.com/mnews/article/092/0002388181?sid=105
https://n.news.naver.com/mnews/article/003/0013449270?sid=102
https://n.news.naver.com/mnews/article/001/0015593497?sid=104
https://n.news.naver.com/mnews/article/277/0005643380?sid=102
https://n.news.naver.com/mnews/article/018/0006101262?sid=105
https://n.news.naver.com/mnews/article/366/0001103929?sid=105


/var/folders/xr/0xbgl9wd3wz914nd58ylmykw0000gp/T/ipykernel_44367/1142016379.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument

In [2]:
demo.close()  # gradio 서버 중지

Closing server running on port: 7860
